In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.signal as signal
import pandas as pd

Load model data

In [2]:
model_name = 'ecmwfsipn'
model_type = 'reforecast'
no_day_change = 5
filepath = '/home/disk/sipn/mcmcgraw/data/VRILE/'
filename = filepath+'{model_name}_{model_type}_SIE_d_SIE_{d_days}day_change_lead_time_ALL_REGIONS_ALL_ENS.csv'.format(model_name=model_name,
                       model_type=model_type,d_days=no_day_change)
SIE_data = pd.read_csv(filename)
SIE_data = SIE_data.dropna()
SIE_data.head()

,Unnamed: 0,I (init date),V (valid date),SIE,lead time (days),d_SIE (V - I),ensemble,region
0,0,1993-01-01,1993-01-03,13.261036,2.0,0.046306,1.0,panArctic
1,1,1993-01-01,1993-01-04,13.268475,3.0,-0.003511,1.0,panArctic
2,2,1993-01-01,1993-01-05,13.230590,4.0,-0.003011,1.0,panArctic
3,3,1993-01-01,1993-01-06,13.224993,5.0,0.020101,1.0,panArctic
4,4,1993-01-01,1993-01-07,13.258025,6.0,0.077908,1.0,panArctic


Select a region for testing 

In [3]:
region_names = SIE_data['region'].dropna().unique().tolist()
region_ind_test = 15
region_name_test = region_names[region_ind_test]
SIE_region_groupby = SIE_data.groupby(['region'])
SIE_by_region = SIE_region_groupby.get_group(region_name_test)

Select only a few lead times

In [4]:
lead_days_select = [7,14,21,28]
f_lead_days_sel = SIE_by_region['lead time (days)'].isin(lead_days_select)
SIE_lead_sel = SIE_by_region.loc[f_lead_days_sel]

Group by month and then lead time, and calculate the 5th percentile

In [18]:
d_SIE_5th_pctile_events = pd.DataFrame(columns=['valid month','lead time (days)',
                                                'd_SIE 5th pctile','region'])
month_str = 'all'
SIE_months = pd.to_datetime(SIE_lead_sel['V (valid date)']).dt.month
SIE_lead_sel['valid date months'] = SIE_months
SIE_month_grouped = SIE_lead_sel.groupby(['valid date months'])#,'lead time (days)'])
SIE_month_anom = SIE_lead_sel.groupby(['valid date months']).transform('mean')
SIE_month_anom = SIE_lead_sel - SIE_lead_sel.groupby(['valid date months']).transform('mean')
SIE_month_anom

/home/disk/sipn/mcmcgraw/anaconda3/envs/VRILE_environment/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,I (init date),SIE,Unnamed: 0,V (valid date),d_SIE (V - I),ensemble,lead time (days),region,valid date months
23220005,NaN,0.006010,-2542.5,NaN,-0.006426,-12.0,-10.5,NaN,NaN
23220012,NaN,0.006010,-2535.5,NaN,0.005013,-12.0,-3.5,NaN,NaN
23220019,NaN,0.006010,-2528.5,NaN,-0.000706,-12.0,3.5,NaN,NaN
23220026,NaN,0.000291,-2521.5,NaN,-0.025487,-12.0,10.5,NaN,NaN
23220216,NaN,0.006010,-2331.5,NaN,-0.000706,-11.0,-10.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...
24767504,NaN,0.021341,2331.5,NaN,-0.008221,11.0,10.5,NaN,NaN
24767694,NaN,-0.265878,2521.5,NaN,-0.035472,12.0,-10.5,NaN,NaN
24767701,NaN,-0.199810,2528.5,NaN,0.039370,12.0,-3.5,NaN,NaN
24767708,NaN,-0.203737,2535.5,NaN,-0.021067,12.0,3.5,NaN,NaN


Now, plot VRILE threshold as a function of 